# Ben is my best friend

### Load code libraries that we'll need

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

### Store all the URLs for all the states using this website format

In [2]:
URLS = {
  'KY': 'https://www.kentuckygolf.com/golfcourses/results.cfm?startrow=',
  'TN': 'https://www.golftennessee.com/golfcourses/results.cfm?startrow=',
}

### Helper functions

In [3]:
# Given a direct url to a single course's information page, find and return the phone number.
def get_course_phone(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text)
    elt = soup.find_all('h4')[0]  # The first h4 is the one we want
    lines = elt.text.splitlines()
    for line in lines:
        line = line.strip()  # Remove whitespace
        if line.startswith('Phone:'):
            # Will look like: 'Phone: 270-646-4653'
            return line.split()[1]  # Return everything after the first space

In [4]:
# Return data for (up to) 20 golf courses on the given website, starting with course number "row_start"
def get_page(base_url, row_start):
    url = base_url + str(row_start)
    response = requests.get(url)

    # From 'https://www.kentuckygolf.com/golfcourses/results.cfm?startrow='
    # extract the substring, 'https://www.kentuckygolf.com/golfcourses/'
    # We'll use this to fetch individual course phone numbers later.
    root_url = url.split('results.cfm')[0]

    # Find all the golf course information blocks, which are identified by
    # an <h4> tag with an <a> nested inside it (which has the course name
    # and a link to its website). We are looking for things like:
    # <h4><a href="../golfcourses/barren-river-lake-golf-course">Barren River Lake State Resort Park</a></h4>
    soup = BeautifulSoup(response.text)
    headers = soup.find_all('h4')
    filtered_headers = []
    for elt in headers:
        if elt.find('a'):
            filtered_headers.append(elt)

    results = []
    for elt in filtered_headers:
        print(len(results) + row_start, end=' ')
        name = elt.text.strip()  # Remove whitespace
        url = elt.find('a').attrs['href']  # Will look like '../golfcourses/barren-river-lake-golf-course'
        url = root_url + url.split('/golfcourses/')[1]  # Extract 'barren-river-lake-golf-course'
        address = elt.findNext('p').findNext('p').text  # Address is listed two <p> tags later
        address = ' '.join(address.split())  # Remove interstitial white space
        phone = get_course_phone(url)  # Load the course page and find the phone number
        results.append([name, address, phone, url])
    print()
    return results

### Fetch each state's data, writing each to its own file

In [5]:
for state, url in URLS.items():
    print(state)
    results = []
    row = 1
    while True:
        print("Fetching 20 courses starting with number", row)
        new_results = get_page(url, row)
        results.extend(new_results)
        # Save data as we go, so we can look at the partially-complete CSV files
        # as the code is doing its thing. So we can flag problems without having
        # to wait for the whole thing to finish.
        data = pd.DataFrame(results, columns=['name', 'address', 'phone', 'url'])
        data.to_csv('courses_' + state + '.csv', index=False)
        if len(new_results) < 20:
            break
        row += 20
    print(len(new_results), 'pulled for', state)
    print()

KY
Fetching 20 courses starting with number 1
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
Fetching 20 courses starting with number 21
21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 
Fetching 20 courses starting with number 41
41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 
Fetching 20 courses starting with number 61
61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 
Fetching 20 courses starting with number 81
81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 
Fetching 20 courses starting with number 101
101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 
Fetching 20 courses starting with number 121
121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 
Fetching 20 courses starting with number 141
141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 
Fetching 20 courses starting with number 161
161 162 163 164 165 166 167 168 169 170 171 172 173 174 17